In [68]:
import json
import sqlite3
import pandas as pd

In [69]:
data = []
with open('kindle_reviews.json') as json_file:
    for idx, line in enumerate(json_file):
        data.append(json.loads(line))
    
print(len(data))

982619


In [70]:
import os

print(os.getcwd())

/Users/hayleengn/Desktop/emse6586


In [77]:
conn = sqlite3.connect('kindle_reviews.db')
cursor = conn.cursor()

conn.commit()
conn.close()

In [78]:
cursor.execute("DROP TABLE IF EXISTS kindle")

ProgrammingError: Cannot operate on a closed database.

In [ ]:
cursor.execute("""CREATE TABLE IF NOT EXISTS kindle (
    reviewerID  TEXT,
    asin TEXT,
    reviewerName  TEXT,
    helpful   TEXT,
    reviewText  TEXT,
    overall    REAL,
    summary    TEXT,
    unixReviewTime INTEGER,
    reviewTime TEXT)"""
);

In [ ]:
# Insert the JSON data into the table
for item in data:
    if "reviewerName" not in item:
        item["reviewerName"] = ""
    values = [str(value) for value in item.values()]
    values_tuple = tuple(values)
    
    cursor.execute("INSERT INTO kindle (reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)", values_tuple) 

In [49]:
print(data[0])

{'reviewerID': 'A1F6404F1VG29J', 'asin': 'B000F83SZQ', 'reviewerName': 'Avidreader', 'helpful': [0, 0], 'reviewText': "I enjoy vintage books and movies so I enjoyed reading this book.  The plot was unusual.  Don't think killing someone in self-defense but leaving the scene and the body without notifying the police or hitting someone in the jaw to knock them out would wash today.Still it was a good read for me.", 'overall': 5.0, 'summary': 'Nice vintage story', 'unixReviewTime': 1399248000, 'reviewTime': '05 5, 2014'}


In [57]:
conn.commit()
conn.close()

In [58]:
conn = sqlite3.connect('kindle_reviews.db')

query = "SELECT * FROM kindle"

# Read the data into a DataFrame
df = pd.read_sql_query(query, conn)
conn.close()

DatabaseError: Execution failed on sql 'SELECT * FROM kindle': database is locked